# Python и машинное обучение: нейронные сети и компьютерное зрение

## Модуль 4. Классификация фото и Аугментация данных

- Загрузка и организация датасета для классификации
- Создание сверточной нейронной сети "с ноля"
- Раширение набора данных с помощью аугментации

In [ ]:
!pip install torchinfo torchmetrics

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets,transforms

from torch.nn.functional import normalize

from torchinfo import summary
from torchmetrics import Accuracy, AUROC

from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F

from PIL import Image

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
device = "cuda" if torch.cuda.is_available() else \
    "mps" if torch.backends.mps.is_built() else "cpu"
device

## Датасет

Будет использоваться набор данных cats vs. dogs:
`https://www.kaggle.com/c/dogs-vs-cats/data`.

Он состоит из изображений среднего размера, в формате JPEG, примерно таких:

![cats_vs_dogs_samples](https://s3.amazonaws.com/book.keras.io/img/ch5/cats_vs_dogs_samples.jpg)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Копируем файлы в runtime (иначе обучение будет идти ооочень медленно).

In [ ]:
!cp -r /content/drive/MyDrive/datasets/cats_and_dogs_small /content/data

In [ ]:
!ls -al /content/data

Датасет уже разбит на обучающую, валидационную и тестовую выборки. Тестовая выборка нами использоваться не будет. Ниже создаем необходимые переменные и убеждаемся, что все файлы на местах.

In [ ]:
import os, shutil

base_dir = '/home/ise/Documents/datasets/cats_and_dogs_small'
# base_dir = '/content/data'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total training dog images:', len(os.listdir(train_dogs_dir)))
print('total validation cat images:', len(os.listdir(validation_cats_dir)))
print('total validation dog images:', len(os.listdir(validation_dogs_dir)))

In [ ]:
IMAGE_WIDTH=150
IMAGE_HEIGHT=150

IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)

# простой трансформинг:
data_transforms = transforms.Compose([
    transforms.Resize(size=IMAGE_SIZE), # делаем все картинки квадратными
    transforms.ToTensor(), # преобразуем в тензор 
])

train_data = datasets.ImageFolder(root=train_dir, # target folder of images
                                  transform=data_transforms, # transforms to perform on data (images)
                                  target_transform=None) # transforms to perform on labels (if necessary)
val_data = datasets.ImageFolder(root=validation_dir, transform=data_transforms)

print(f"Train data:\n{train_data}\nTest data:\n{val_data}")

In [ ]:
# названия классов
class_names = train_data.classes
print("Class names: ",class_names)

# ...в виде словаря
class_dict = train_data.class_to_idx
print("Class names as a dict: ",class_dict)


In [ ]:
ix_random_image = np.random.choice(len(train_data))

img, label = train_data[ix_random_image]
print(f"Image tensor:\n{img}")
print(f"Image shape: {img.shape}")
print(f"Image datatype: {img.dtype}")
print(f"Image label: {label}={class_names[label]}")
display(transforms.ToPILImage()(img))

Создаем генераторы и разделяем загрузку данных на несколько ядер нашего/наших CPU.

In [ ]:
NUM_WORKERS = os.cpu_count()
print(f'Cores: {NUM_WORKERS}')

BATCH_SIZE = 20

train_gen = DataLoader(dataset=train_data, 
                              batch_size=BATCH_SIZE,
                              num_workers=NUM_WORKERS,
                              shuffle=True) 

val_gen = DataLoader(dataset=val_data, 
                             batch_size=BATCH_SIZE, 
                             num_workers=NUM_WORKERS, 
                             shuffle=False)


In [ ]:
# визуализируем батч
images, labels = next(iter(train_gen))
images_np = images.numpy()

print(images.shape)

fig = plt.figure(figsize=(20, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20//2, idx+1, xticks=[], yticks=[])
    ax.imshow(np.transpose(images_np[idx], (1,2,0)))
    ax.set_title(class_names[labels[idx].item()])

## Строим нейросеть

Делаем ее как композицию 4-х слоев ```Conv2D``` и ```MaxPooling2D```, емкость карт признаков - 32, 64, 128, 128. Перед выходом добавляем два полносвязных ```Dense``` слоя. В качестве функции активации выбираем (конечно) ```relu```, в последнем слое функцию активаци не делаем.

Такая сеть использовалась Ф. Шолле в его книге про фреймворк Keras.


In [ ]:
class ImageClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, # принимает трехканальное изображение 3, 150, 150
                     out_channels=32, # отдает карту признаков на 32 канала: 32, 148, 148 
                     kernel_size=3, # ядро размера 3х3
                     padding=0, 
                     bias=False),
            nn.ReLU(),
            nn.MaxPool2d(2)) # отдает карту признаков на 32 канала: 32, 74, 74
            
        # ваш код здесь
            
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features= ? , out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=2),
        )
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.classifier(x)
        return x

model = ImageClassifier().to(device)
print(model)

summary(model,
        input_size=images.shape,
        col_names=["input_size", "output_size", "num_params"],
        device=device
       )

Выбираем функцию потерь ```crossentropy```, оптимизатор - ```RMSprop```, метрика - точность (```accuracy```).

In [ ]:
criterion = nn.CrossEntropyLoss()

y_preds = model(images.to(device))
print(y_preds)
print(labels.to(device))

loss = criterion(y_preds, labels.to(device))
print("Loss", loss.data.item())

print("Acccuracy", (y_preds.argmax(dim=1) == labels.to(device)).float().mean())



In [ ]:
def train_batches(model,
                  train_generator,
                  valid_generator,
                  batch_size=20, epochs=40, report_positions=20, **kwargs):

    results = {'epoch_count': [], 'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

    # прогоняем данные по нейросети
    for epoch in range(epochs):
        model.train()

        train_loss = valid_loss = 0.0;
        train_correct = valid_correct = 0.0

        for X_batch, y_batch in train_generator:

            X_batch = X_batch.to(device); y_batch = y_batch.to(device)

            y_preds = model(X_batch) 
            loss = criterion(y_preds, y_batch) 

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.data.item()
            train_correct += (y_preds.argmax(dim=1) == y_batch).float().sum()

        train_loss /= len(train_generator.dataset)
        train_acc = 100 * train_correct / len(train_generator.dataset)    

        # Валидацию тоже делаем по батчам
        model.eval()

        for valid_batches, (X_val_batch, y_val_batch) in enumerate(valid_generator):
            X_val_batch = X_val_batch.to(device); y_val_batch = y_val_batch.to(device)
            y_batch_preds = model(X_val_batch)
            loss = criterion(y_batch_preds, y_val_batch)

            valid_loss += loss.data.item()
            valid_correct += (y_batch_preds.argmax(dim=1) == y_val_batch).float().sum()

        valid_loss /= len(valid_generator.dataset)
        valid_acc = 100 * valid_correct / len(valid_generator.dataset)

        results['epoch_count'] += [epoch]
        results['train_loss'] += [ train_loss ]
        results['train_acc'] += [ float(train_acc) ]
        results['val_loss'] += [ valid_loss ]
        results['val_acc'] += [ float(valid_acc) ]

        if epoch % (epochs // report_positions) == 0 or epochs<50:
            print(f"Epoch: {epoch+1:4.0f} | Train Loss: {train_loss:.5f}, "+\
                  f"Accuracy: {train_acc:.2f}% | \
            Validation Loss: {valid_loss:.5f}, Accuracy: {valid_acc:.2f}%")

    return results

# рисовалка графиков
def plot_results(results):

    fig, axs = plt.subplots(1,2)

    fig.set_size_inches(10,3)

    for i, loss_acc in enumerate(['loss', 'acc']):
        for train_val in ['train', 'val']:
            axs[i].plot(results['epoch_count'], results[f'{train_val}_{loss_acc}'], label=f'{loss_acc} {train_val}')

        axs[i].legend()

    plt.show()

In [ ]:
model = ImageClassifier().to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=3e-4)

results = train_batches(model,
                        train_gen,
                        val_gen, epochs=30, )

plot_results(results)

Очевидно переобучение: показатели на обучающей выборке гораздо лучше, чем на контрольной. Что делать?

## Используем data augmentation

Переобучение вызвано ограниченным набором образцов - для модели явно мало 2000 картинок. Что поможет нам расширить обучающий набор? Если мы применим к каждой картинке некое преобразование (например, будем ее "зеркалить" по горизонтали) - это уже увеличит объем выборки вдвое. Можно также применять другие преобразования, которые оставят картинку узнаваемой человеком (глядя на нее любой сможет сказать: "смотрите, это кот!"), но при этом сделают ее "новой" для нейросети. Например, можно картинку немного увеличивать, слегка поворачивать, чуть-чуть сдвигать.

Такие преобразования называют "аугментацией данных" - расширением обучающего набора на базе имеющегося, таким образом, что сохраняется принадлежность образцов к исходным классам.

Вот пример таких преобразований:
- вращаем в пределах 40*
- сдвигаем по ширине на 20%
- сдвигаем по высоте на 20%
- делаем трапецевидный сдвиг на 20%
- увеличиваем на 20%
- делаем горизонтальное зеркалирование

Создадим новую нейросеть: у нее такая же архитектура как у предыдущей, но мы после комбинации ```Conv2D-MaxPooling2D``` добавили еще слой - прореживатель (```Dropout```).

In [ ]:
# ваш код здесь


Запустим обучение: увеличим количество эпох до 100, все остальные параметры оставим как прежде. Обучение в Google Colab может занять около часа.